In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import itertools
import math
import pickle
import subprocess
from os import path
from tempfile import TemporaryDirectory

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.stats import chi2, entropy

In [3]:
def data_path(set_number):
    return 'data/sample-laser-radar-measurement-data-{}.txt'.format(set_number)

In [4]:
#
# Run the UnscentedKF program with the given arguments and return the output.
#
def run(set_number, use_laser, use_radar, std_a, std_yawdd, lambd):
    with TemporaryDirectory() as tmp:
        output_path = path.join(tmp, 'output.txt')
        command = [
          'build/UnscentedKF',
          data_path(set_number),
          output_path,
          str(use_laser).lower(),
          str(use_radar).lower(),
          str(std_a),
          str(std_yawdd),
          str(lambd),
          'true' # tweak output for reading by this tool
        ]
        result = subprocess.run(
          command,
          stdout=subprocess.PIPE,
          stderr=subprocess.PIPE,
          universal_newlines=True
        )
        with open(output_path, 'r') as output_file:
            return result, output_file.readlines()

In [5]:
#
# Since we've already computed the RMSEs, just extract them from stdout.
#
def extract_rmse(result):
    if result.returncode != 0:
        return None
    lines = result.stdout.splitlines()
    rmse_start = lines.index('Accuracy - RMSE:') + 1
    rmse_end = rmse_start + 4
    return [float(rmse) for rmse in lines[rmse_start:rmse_end]]

SENSOR_INDEX = 11
NIS_INDEX = 12

def check_output_header_line(line):
    fields = line.split()
    assert len(fields) == 13
    assert fields[SENSOR_INDEX] == 'sensor'
    assert fields[NIS_INDEX] == 'NIS'

def extract_nis(output):
    check_output_header_line(output[0])
    rows = [
        output[i].split()
        for i in range(1, len(output))
    ]
    return [
        [float(row[NIS_INDEX]) for row in rows if row[SENSOR_INDEX] == sensor]
        for sensor in ['L', 'R'] 
    ]

In [6]:
#
# This is the NIS score test suggested in the lecture: find the expected NIS
# based on the 95th percentile of the Chi^2 distribution with the relevant
# number of degrees of freedom (e.g. NIS ~ 7.8 for 3 DoF). Then find the
# percentage of observed NIS scores that are below that threshold. If the answer
# is close to 95%, that suggests that the filter is consistent.
#
def find_nis_quantile(nis_scores, df, quantile=0.95):
    if len(nis_scores) == 0: return float('nan')
    threshold = chi2.ppf(quantile, df)
    return sum(nis < threshold for nis in nis_scores) / len(nis_scores)

# Check: this number should be about 0.95
find_nis_quantile(chi2.rvs(2, size=1000), 2)

0.94499999999999995

In [7]:
#
# Compute the Kullback–Leibler divergence between the empirical distribution
# of NIS scores and the Chi^2 distribution with the given number of degrees of
# freedom. If the empirical distribution of the NIS scores is close to the
# expected Chi^2 distribution, so the filter is consistent, then the returned
# KL-div will be small.
#
def find_nis_kl_div(nis_scores, df, num_bins=21):
    if len(nis_scores) == 0: return float('nan')
    if not np.all(np.isfinite(nis_scores)): return float('nan')
    # Pick the bins so that we expect an equal mass in each bin, according to
    # the Chi^2 distribution with the appropriate number of degrees of freedom.
    xs = np.linspace(0, 1, num_bins)
    pk, _ = np.histogram(nis_scores, bins=chi2.ppf(xs, df))
    pk = pk.astype(np.double) / np.sum(pk)
    qk = np.diff(xs)
    return entropy(pk, qk)

# Check: this number should be near zero.
find_nis_kl_div(chi2.rvs(2, size=1000), 2)

0.010197440303248956

In [8]:
# From http://stackoverflow.com/a/40623158/2053820
def dict_product(dicts):
    """
    >>> list(dict_product(dict(number=[1,2], character='ab')))
    [{'character': 'a', 'number': 1},
     {'character': 'a', 'number': 2},
     {'character': 'b', 'number': 1},
     {'character': 'b', 'number': 2}]
    """
    return (dict(zip(dicts, x)) for x in itertools.product(*dicts.values()))

In [9]:
def evaluate(params):
    result, output = run(**params)
    if result.returncode == 0:
        rmse = extract_rmse(result)
        laser_nis_scores, radar_nis_scores = extract_nis(output)
    else:
        rmse = [float('inf')] * 4
        laser_nis_scores = []
        radar_nis_scores = []

    laser_df = 2
    radar_df = 3
    return {
        'px_rmse': rmse[0],
        'py_rmse': rmse[1],
        'vx_rmse': rmse[2],
        'vy_rmse': rmse[3],
        'laser_nis_kl_div': find_nis_kl_div(laser_nis_scores, laser_df),
        'laser_nis_quantile': find_nis_quantile(laser_nis_scores, laser_df),
        'radar_nis_kl_div': find_nis_kl_div(radar_nis_scores, radar_df),
        'radar_nis_quantile': find_nis_quantile(radar_nis_scores, radar_df)
    }

In [ ]:
# First grid:
#         'std_a': np.linspace(0.1, 10, 20).tolist(),
#         'std_yawdd': (math.pi / np.linspace(0.1, 10, 20)).tolist(),
#         'lambd': np.linspace(-10, 10, 40).tolist()
# Yielded only 30 out of 32000 points that met spec.
# Narrowed the ranges for the second grid.
# Then added a few more std_a and std_yawdd points, because the
# best ones were near the edge of the space.

In [105]:
RESULTS_FILE = 'data/search.pickle'
def search():
    if path.isfile(RESULTS_FILE):
        with open(RESULTS_FILE, 'rb') as f:
            results = pickle.load(f)
    else:
        results = {}

    keys = {
        'set_number': [1, 2],
        'use_laser': [True],
        'use_radar': [True],
        'std_a': np.concatenate([
            np.linspace(0.2, 0.5, 10),
            np.linspace(0.3, 0.5, 10),
            np.linspace(0.5, 2.0, 20)]).tolist(),
        'std_yawdd': (math.pi / np.concatenate([
                np.linspace(6.0, 8.0, 10),
                np.linspace(4.0, 6.0, 20)])).tolist(),
        'lambd': np.linspace(-6, -1, 20).tolist()
    }

    for key in dict_product(keys):
        frozen_key = frozenset(key.items())
        if frozen_key in results:
            continue

        results[frozen_key] = evaluate(key)

        with open(RESULTS_FILE, 'wb') as f:
            pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

search()

In [106]:
def load_results():
    with open(RESULTS_FILE, 'rb') as f:
        return pickle.load(f)

# Pair up the dataset 1 and dataset 2 results --- we want to
# consider both for each set of parameters.
def group_results(results):
    grouped_results = {}
    for key, value in results.items():
        key_dict = dict(key)
        set_number = key_dict.pop('set_number')
        key_without_set_number = frozenset(key_dict.items())
        if key_without_set_number in grouped_results:
            grouped_results[key_without_set_number][set_number] = value
        else:
            grouped_results[key_without_set_number] = {set_number: value}
    return grouped_results

GROUPED_RESULTS = group_results(load_results())
len(GROUPED_RESULTS)

21460

### Apply max RMSE constraint

In [107]:
def make_single_rmse_vector(values):
    return np.array([
        values['px_rmse'],
        values['py_rmse'],
        values['vx_rmse'],
        values['vy_rmse'],
    ])

def make_rmse_vector(values):
    return np.concatenate([
        make_single_rmse_vector(values[1]),
        make_single_rmse_vector(values[2])
    ])

MAX_RMSES = {
    1: {
        'px_rmse': 0.09,
        'py_rmse': 0.09,
        'vx_rmse': 0.65,
        'vy_rmse': 0.65
    },
    2: {
        'px_rmse': 0.20,
        'py_rmse': 0.20,
        'vx_rmse': 0.55,
        'vy_rmse': 0.55
    }
}
MAX_RMSE_VECTOR = make_rmse_vector(MAX_RMSES)

def is_result_in_spec(value):
    rmse_vector = make_rmse_vector(value)
    if np.any(np.isnan(rmse_vector)):
        return False
    return np.all(rmse_vector <= MAX_RMSE_VECTOR)

# We need it to be in spec for both dataset 1 and dataset 2.  
def find_results_in_spec(results):
    return {
        key: value
        for key, value in results.items()
        if is_result_in_spec(value)
    }

def make_single_nis_vector(values, consistency_score):
    return np.array([
        values['laser_nis_' + consistency_score],
        values['radar_nis_' + consistency_score]
    ])

def make_nis_vector(values, consistency_score):
    return np.concatenate([
        make_single_nis_vector(values[1], consistency_score),
        make_single_nis_vector(values[2], consistency_score)
    ])

def make_result_vector(values, consistency_score):
    return np.concatenate([
        make_single_rmse_vector(values[1]),
        make_single_nis_vector(values[1], consistency_score),
        make_single_rmse_vector(values[2]),
        make_single_nis_vector(values[2], consistency_score)
    ])

# v1 dominates v2 if all entries of v1 are smaller
def dominates(v1, v2):
    return np.all(v1 < v2)

def find_non_dominated(results, consistency_score):
    keys = list(results.keys())
    vectors = [
        make_result_vector(results[keys[i]], consistency_score)
        for i in range(len(keys))
    ]
    return {
        keys[i]: results[keys[i]]
        for i in range(len(keys))
        if not any(
            dominates(vectors[j], vectors[i])
            for j in range(len(keys)))
    }

RESULTS_IN_SPEC = find_results_in_spec(GROUPED_RESULTS)
len(RESULTS_IN_SPEC)

4842

In [108]:
NON_DOMINATED_KL_DIV_RESULTS = \
    find_non_dominated(RESULTS_IN_SPEC, 'kl_div')
len(NON_DOMINATED_KL_DIV_RESULTS)

4834

In [112]:
dom = list(set(RESULTS_IN_SPEC) - set(NON_DOMINATED_KL_DIV_RESULTS))[0]
domv = make_result_vector(RESULTS_IN_SPEC[dom], 'kl_div')
(dom, domv)

(frozenset({('lambd', -1.526315789473684),
            ('std_a', 0.3666666666666667),
            ('std_yawdd', 0.552687596464871),
            ('use_laser', True),
            ('use_radar', True)}),
 array([ 0.0768951 ,  0.0842682 ,  0.596263  ,  0.57953   ,  0.43099722,
         0.25169172,  0.195603  ,  0.18771   ,  0.290216  ,  0.523151  ,
         0.3310879 ,  0.81138413]))

In [111]:
[
    (key, make_result_vector(value, 'kl_div'))
    for key, value in RESULTS_IN_SPEC.items()
    if dominates(make_result_vector(value, 'kl_div'), domv)
]

[(frozenset({('lambd', -2.0526315789473686),
             ('std_a', 0.3888888888888889),
             ('std_yawdd', 0.552687596464871),
             ('use_laser', True),
             ('use_radar', True)}),
  array([ 0.0767982 ,  0.0839924 ,  0.596012  ,  0.579288  ,  0.4298233 ,
          0.24790862,  0.194879  ,  0.187557  ,  0.28153   ,  0.50407   ,
          0.32986809,  0.76055473]))]

In [113]:
def average_rmse(rmses):
    return np.sqrt(np.dot(rmses, rmses) / len(rmses))
    
def plot_in_parameter_space(results, size_scale=30):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    data = np.array([
        [
            dict(key)['std_a'],
            dict(key)['std_yawdd'],
            dict(key)['lambd'],
            size_scale * np.mean(make_nis_vector(value, 'kl_div')),
            average_rmse(make_rmse_vector(value))
        ]
        for key, value in results.items()
    ])
    p = ax.scatter(data[:,0], data[:,1], data[:,2],
                   s=data[:,3], c=data[:,4], cmap='cool')
    ax.set_xlabel('std_a')
    ax.set_ylabel('std_yawdd')
    ax.set_zlabel('lambda')
    fig.colorbar(p)
    
plot_in_parameter_space(NON_DOMINATED_KL_DIV_RESULTS)

In [114]:
def plot_in_objective_space(results, consistency_score, pareto_only):
    keys = list(results.keys())
    values = [results[key] for key in keys]
    data = np.array([
        [
            np.mean(make_nis_vector(value, consistency_score)),
            average_rmse(make_rmse_vector(value))
        ]
        for value in values
    ])
    
    if pareto_only:
        indexes = [
            i
            for i in range(len(keys))
            if not any (
                np.all(data[j,:] < data[i,:])
                for j in range(len(keys))
            )
        ]
        for i in indexes:
            key_i = dict(keys[i])
            print(key_i['std_a'], key_i['std_yawdd'], key_i['lambd'],
                  data[i,0], data[i,1])
    else:
        indexes = range(len(keys))        
    
    fig, ax = plt.subplots()
    ax.scatter(data[indexes,0], data[indexes,1])
    
    for i in indexes:
        key = dict(keys[i])
        text = "{std_a:.3f},{std_yawdd:.3f},{lambd:.3f}".format(**key)
        ax.annotate(text, (data[i,0], data[i,1]))
    
    plt.xlabel('average NIS ' + consistency_score)
    plt.ylabel('average RMSE')
plot_in_objective_space(NON_DOMINATED_KL_DIV_RESULTS, 'kl_div', pareto_only=True)

0.33333333333333337 0.5235987755982988 -5.473684210526316 0.312495913273 0.36603127876
0.26666666666666666 0.5329487537339828 -4.684210526315789 0.380464991037 0.360401629196
0.26666666666666666 0.5426387310746007 -5.473684210526316 0.320188873935 0.363810472886
0.26666666666666666 0.5329487537339828 -5.473684210526316 0.32571985334 0.363269151905
0.30000000000000004 0.5235987755982988 -4.947368421052632 0.353903201299 0.361273356852
0.3 0.5235987755982988 -4.947368421052632 0.353903201299 0.361273356852
0.23333333333333334 0.552687596464871 -5.473684210526316 0.32841632889 0.363204877381
0.3 0.5235987755982988 -5.2105263157894735 0.340133074897 0.362233943089
0.26666666666666666 0.5329487537339828 -5.2105263157894735 0.341408405975 0.361421288147
0.4666666666666667 0.5235987755982988 -5.2105263157894735 0.305383801662 0.370134445195
0.30000000000000004 0.5235987755982988 -5.2105263157894735 0.340133074897 0.362233943089
0.26666666666666666 0.5329487537339828 -4.947368421052632 0.35779

In [115]:
plot_in_objective_space(RESULTS_IN_SPEC, 'quantile', pareto_only=True)

0.2 0.5851986315510399 -5.2105263157894735 0.938382352941 0.362935120189
0.23333333333333334 0.5739448117135199 -5.2105263157894735 0.940065359477 0.362132642887
0.26666666666666666 0.5739448117135199 -5.473684210526316 0.933382352941 0.366149586484
0.23333333333333334 0.552687596464871 -5.473684210526316 0.935065359477 0.363204877381
0.23333333333333334 0.552687596464871 -5.2105263157894735 0.942565359477 0.361386002991
0.26666666666666666 0.5329487537339828 -4.947368421052632 0.944656862745 0.360608440205
0.30000000000000004 0.5426387310746007 -5.473684210526316 0.934248366013 0.365533502719
0.26666666666666666 0.5329487537339828 -5.473684210526316 0.934656862745 0.363269151905
0.23333333333333334 0.552687596464871 -4.684210526315789 0.945065359477 0.360582828011
0.3 0.5426387310746007 -5.473684210526316 0.934248366013 0.365533502719
0.3 0.5329487537339828 -5.473684210526316 0.934248366013 0.364848910003
0.30000000000000004 0.5329487537339828 -5.473684210526316 0.934248366013 0.36484

In [117]:
def find_weighted_score(value, rmse_weight,
                        consistency_weight, consistency_score):
    consistency = np.mean(make_nis_vector(value, consistency_score))
    # Want to be as close to 0.95 as possible.
    if consistency_score == 'quantile':
        consistency = np.abs(consistency - 0.95)
    return \
        rmse_weight * average_rmse(make_rmse_vector(value)) + \
        consistency_weight * consistency
    
def find_top_results(
    results, n,
    rmse_weight=1,
    consistency_weight=0.1,
    consistency_score='kl_div'):
    sorted_results = sorted(
        results.items(),
        key=lambda pair: find_weighted_score(
            pair[1], rmse_weight, consistency_weight, consistency_score))
    return [
        (value, dict(key))
        for key, value in sorted_results
    ][0:n]
find_top_results(NON_DOMINATED_KL_DIV_RESULTS, 5)

[({1: {'laser_nis_kl_div': 0.36708907635530003,
    'laser_nis_quantile': 1.0,
    'px_rmse': 0.0790429,
    'py_rmse': 0.0895979,
    'radar_nis_kl_div': 0.24062287776881455,
    'radar_nis_quantile': 0.81862745098039214,
    'vx_rmse': 0.60153,
    'vy_rmse': 0.585284},
   2: {'laser_nis_kl_div': 0.18545954058779751,
    'laser_nis_quantile': 0.96999999999999997,
    'px_rmse': 0.181726,
    'py_rmse': 0.187455,
    'radar_nis_kl_div': 0.57246212918647921,
    'radar_nis_quantile': 0.98999999999999999,
    'vx_rmse': 0.292168,
    'vy_rmse': 0.415699}},
  {'lambd': -5.2105263157894735,
   'std_a': 0.26666666666666666,
   'std_yawdd': 0.5329487537339828,
   'use_laser': True,
   'use_radar': True}),
 ({1: {'laser_nis_kl_div': 0.37602558621686799,
    'laser_nis_quantile': 1.0,
    'px_rmse': 0.0778441,
    'py_rmse': 0.0892303,
    'radar_nis_kl_div': 0.23562352598140654,
    'radar_nis_quantile': 0.8202614379084967,
    'vx_rmse': 0.599763,
    'vy_rmse': 0.584357},
   2: {'laser_nis

In [14]:
# could do a grid, or we could use an optimiser...
# but I guess it's multiobjective
# and it's pretty quick.
# probably simpler to just grid it out

# record for each param setting:
#   for each dataset:
#     RMSEs
#     NIS quantile
#     NIS KLDIV
#     or a failure code
# then I guess we want another script to thin it out with:
#  - not a numerical failure
#  - meets spec in terms of RMSEs on DS1 and DS2
#  - not dominated in terms of the 8 RMSEs and 2 KLdivs (pareto)
#
# can't plot 10 dimensions, but can project into mean RMSE and mean KLdiv space
# and we only have 3 real params, so we can do a 3d plot

# odd that we didn't get any dominated solutions... maybe a bug
# but maybe we should instead do pareto on the 2d plot. That looks like we could do
# some pareto analysis to get a small number of solutions.